In [ ]:
import descarteslabs as dl
from datetime import datetime
from shapely.geometry import Point
import osgeo
from osgeo import ogr, osr

In [90]:
def make_geojson(bounding_box): 
    minx = bounding_box[0]
    miny = bounding_box[1]
    maxx = bounding_box[2]
    maxy = bounding_box[3]

    aoi = {"coordinates": [[            
        [miny, minx],
        [miny, maxx],
        [maxy, maxx],
        [maxy, minx],
        [miny, minx]]], 
       "type": "Polygon"}

    return aoi
    

In [91]:
def buffer_point(coords):
    point = Point(coords[0], coords[1])
    buffered_point = point.buffer(.083)
    geo = make_geojson(buffered_point.bounds)
    return geo
    

In [103]:
def buffer_point(coords):
    dl.raster.dltile_from_latlon(coords[0],coords[1], resolution= 1024, tilesize=1024)

In [98]:
def search_products(date, geometry):
    from pprint import pprint 
    images = dl.metadata.search(
        ['landsat:LC08:01:RT:TOAR', 'sentinel-2:L1C'],
        geom= geometry, 
        start_datetime=date, 
        end_datetime= datetime.today().isoformat()
        )
    return [image['id'] for image in images['features']]

In [99]:
def download_imagery(ids, geometry):
    for image_id in ids: 
        print image_id
        dl.raster.raster(
                        image_id,
                        bands=['red', 'green', 'blue', 'alpha'],
                        scales=[[0,4000], [0, 4000], [0, 4000], None],
                        data_type='Byte',
                        cutline=geometry,
                        save= True,
                        outfile_basename=image_id)

In [100]:
def find_imagery(fire_name, date, coords, footprint=None):
    # Where name is a str, coords is an array of [lat,long], and footprint is an optional geometry
    if footprint:
        image_ids = search_products(date,footprint)
        download_imagery(image_ids, footprint)
    else:
        image_ids = search_products(date, buffer_point(coords))
        download_imagery(image_ids, buffer_point(coords))
    

Driver Code

In [101]:
find_imagery('Test Fire', '2018-04-25',[39.143338, -75.747761])

landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_014033_20180428_20180428_01_RT_v1
sentinel-2:L1C:2018-04-26_18SVJ_99_S2B_v1


In [ ]:
# With lat,long
search_products('2018-04-25', buffer_point([39.143338, -75.747761]))

In [ ]:
# With footprint as geojson
maryland = dl.places.shape('north-america_united-states_maryland')
search_products('2018-04-25', maryland)